Determine the highest paying job for each year (annually and hourly)

Pay annually and Pay hourly specify if a job is being pay strictly one way or another

In [ ]:
--Annually 
/*
SELECT MAX(Annually_Mean) AS Highest_Annually, Year
FROM Employment_Project..Employment
WHERE Pay_Hourly IS NULL
GROUP BY Year
ORDER BY Year;
*/
--Note: max annually found a value that may correspond to more than one jobs for each year

--Find all the OCC_CODE with the highest annual mean for each year
/*
SELECT OCC_CODE, Year,
       MAX(Annually_Mean) OVER 
       (
           PARTITION BY Year 
           ORDER BY Year 
       ) AS Highest_Annually
FROM Employment_Project..Employment
WHERE Pay_Hourly IS NULL
*/
/*
SELECT OCC_CODE, Year, Annually_Mean AS Highest_Annually
FROM Employment_Project..Employment
WHERE Annually_Mean = (SELECT MAX(Annually_Mean)
                       FROM Employment_Project..Employment
                       WHERE Pay_Hourly IS NULL)

*/
--Note: Find the correct annually amount but not the correct OCC_CODE!!!
/*
SELECT em.OCC_CODE, em.Year, em.Annually_Mean AS Highest_Annually
FROM Employment_Project..Employment em,
  (SELECT MAX(Annually_Mean) AS Highest_Annually, Year
   FROM Employment_Project..Employment
   WHERE Pay_Hourly IS NULL
   GROUP BY Year) max_annual
WHERE em.Annually_Mean = max_annual.Highest_Annually
AND em.Year = max_annual.Year;
*/
/*
--join the table
SELECT em.OCC_CODE, jb.Job_Title, em.Annually_Mean, em.Year
FROM Employment_Project..Employment em
JOIN Employment_Project..Job_type jb
    ON em.OCC_CODE = jb.OCC_CODE
*/

--Find the job title correspond to the max annual. Join job_type and employment table, sort by year
WITH max_annually AS (
    SELECT em.OCC_CODE, em.Year, em.Annually_Mean AS Highest_Annually
    FROM Employment_Project..Employment em,
        (SELECT MAX(Annually_Mean) AS Highest_Annually, Year
         FROM Employment_Project..Employment
         WHERE Pay_Hourly IS NULL
         GROUP BY Year) max_annual
    WHERE em.Annually_Mean = max_annual.Highest_Annually
    AND em.Year = max_annual.Year 
)
SELECT jb.OCC_CODE, jb.Job_Title, jb.OCC_Group, max_annually.Highest_Annually, max_annually.Year 
FROM Employment_Project..Job_type jb
JOIN max_annually
    ON jb.OCC_CODE = max_annually.OCC_CODE



In [ ]:
--Hourly
--Find the highest hourly mean for each year
/*
SELECT MAX(Hourly_Mean) AS Highest_Hourly, Year
FROM Employment_Project..Employment
WHERE Pay_Annually IS NULL
GROUP BY Year
ORDER BY Year;

--Find all the OCC_CODE with the highest hourly mean for each year
SELECT em.OCC_CODE, em.Year, em.Hourly_Mean AS Highest_Hourly
FROM Employment_Project..Employment em,
  (SELECT MAX(Hourly_Mean) AS Highest_Hourly, Year
   FROM Employment_Project..Employment
   WHERE Pay_Annually IS NULL
   GROUP BY Year) max_hour
WHERE em.Hourly_Mean = max_hour.Highest_Hourly
AND em.Year = max_hour.Year;
*/
--Find the job title correspond to the max hourly. Join job_type and employment table, sort by year
WITH max_hourly AS (
    SELECT em.OCC_CODE, em.Year, em.Hourly_Mean AS Highest_Hourly
    FROM Employment_Project..Employment em,
         (SELECT MAX(Hourly_Mean) AS Highest_Hourly, Year
          FROM Employment_Project..Employment
          WHERE Pay_Annually IS NULL
          GROUP BY Year) max_hour
    WHERE em.Hourly_Mean = max_hour.Highest_Hourly
    AND em.Year = max_hour.Year
)
SELECT jb.OCC_CODE, jb.Job_Title, jb.OCC_Group, max_hourly.Highest_Hourly, max_hourly.Year 
FROM Employment_Project..Job_type jb
JOIN max_hourly
    ON jb.OCC_CODE = max_hourly.OCC_CODE




Lowest paying job for each year

In [ ]:
--Annually
--Find the lowest annually mean for each year
SELECT MIN(Annually_Mean) AS Lowest_Annually, Year
FROM Employment_Project..Employment
WHERE Pay_Hourly IS NULL
GROUP BY Year
ORDER BY Year;

--Find the corresponding job title
WITH min_annually AS (
    SELECT em.OCC_CODE, em.Year, em.Annually_Mean AS Lowest_Annually
    FROM Employment_Project..Employment em,
        (SELECT MIN(Annually_Mean) AS Lowest_Annually, Year
         FROM Employment_Project..Employment
         WHERE Pay_Hourly IS NULL
         GROUP BY Year) min_annual
    WHERE em.Annually_Mean = min_annual.Lowest_Annually
    AND em.Year = min_annual.Year 
)
SELECT jb.OCC_CODE, jb.Job_Title, jb.OCC_Group, min_annually.Lowest_Annually, min_annually.Year 
FROM Employment_Project..Job_type jb
JOIN min_annually
    ON jb.OCC_CODE = min_annually.OCC_CODE


In [ ]:
--Hourly
--Find the lowest hourly mean for each year
SELECT MIN(Hourly_Mean) AS Lowest_Hourly, Year
FROM Employment_Project..Employment
WHERE Pay_Annually IS NULL
GROUP BY Year
ORDER BY Year;

--Find the corresponding job title
WITH min_hourly AS (
    SELECT em.OCC_CODE, em.Year, em.Hourly_Mean AS Lowest_Hourly
    FROM Employment_Project..Employment em,
         (SELECT MIN(Hourly_Mean) AS Lowest_Hourly, Year
          FROM Employment_Project..Employment
          WHERE Pay_Annually IS NULL
          GROUP BY Year) min_hour
    WHERE em.Hourly_Mean = min_hour.Lowest_Hourly
    AND em.Year = min_hour.Year
)
SELECT jb.OCC_CODE, jb.Job_Title, jb.OCC_Group, min_hourly.Lowest_Hourly, min_hourly.Year 
FROM Employment_Project..Job_type jb
JOIN min_hourly
    ON jb.OCC_CODE = min_hourly.OCC_CODE


Amount of employment changes for each jobs (group by occ\_group and job\_title, only consider jobs that is reported in all years)

In [ ]:
--join tables together to make sure that jobs are listed in all 4 years
    --Join table
/*
SELECT em.OCC_CODE, jb.Job_Title, em.Total_Employment, em.Year 
FROM Employment_Project..Employment em
JOIN Employment_Project..Job_type jb
    ON em.OCC_CODE = jb.OCC_CODE
*/

--finding yearly difference
/*
SELECT Year, OCC_CODE, Total_Employment,
       LAG(Total_Employment) OVER ( 
           PARTITION BY OCC_CODE
           ORDER BY Year) AS Employment_Previous_Year,
       Total_Employment - LAG(Total_Employment) OVER ( 
           PARTITION BY OCC_CODE
           ORDER BY Year) AS Year_Difference
FROM Employment_Project..Employment;
*/
--Now, what to find the job name correspond to the code
WITH annual_change AS(
    SELECT Year, OCC_CODE, Total_Employment,
           LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Employment_Previous_Year,
           Total_Employment - LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Year, annual_change.Total_Employment,
       annual_change.Employment_Previous_Year, annual_change.Yearly_Difference
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE

Job that increase (and decrease) in employment  the most throughout the years

In [ ]:
--Job that increase the most
WITH annual_change AS(
    SELECT Year, OCC_CODE, Total_Employment,
           LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Employment_Previous_Year,
           Total_Employment - LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Total_Employment,
       annual_change.Employment_Previous_Year, annual_change.Yearly_Difference AS Highest_Employment_Increase
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE
    WHERE annual_change.Yearly_Difference = (SELECT MAX(Yearly_Difference)
                                             FROM annual_change)

In [ ]:
--Job that decrease the most
WITH annual_change AS(
    SELECT Year, OCC_CODE, Total_Employment,
           LAG(Total_Employment) OVER ( 
           PARTITION BY OCC_CODE
           ORDER BY Year) AS Employment_Previous_Year,
       Total_Employment - LAG(Total_Employment) OVER ( 
           PARTITION BY OCC_CODE
           ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Total_Employment,
       annual_change.Employment_Previous_Year, annual_change.Yearly_Difference AS Highest_Employment_Decrease
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE
    WHERE annual_change.Yearly_Difference = (SELECT MIN(Yearly_Difference)
                                             FROM annual_change)

Do jobs that payed the highest see any changes in employment?

In [ ]:
--highest paying job
WITH annual_change AS(
    SELECT Year, OCC_CODE, Total_Employment,
           LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Employment_Previous_Year,
           Total_Employment - LAG(Total_Employment) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Year, annual_change.Total_Employment,
       annual_change.Employment_Previous_Year, annual_change.Yearly_Difference
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE
    --Anesthesiologists for 2018-2020 (cardiologist only got reported in 2021)
    WHERE annual_change.OCC_CODE = 291211 
--lowest paying job

The amount of annual wages changing for each jobs in between each years (jobs must be reported in all years)

In [ ]:
WITH annual_change AS(
    SELECT Year, OCC_CODE, Annually_Mean,
           LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Wage_Previous_Year,
           Annually_Mean - LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
    WHERE Pay_Hourly IS NULL
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Year, annual_change.Annually_Mean,
       annual_change.Wage_Previous_Year, annual_change.Yearly_Difference AS Wage_Difference
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE

Job that has the most increase (or decrease) in wages

In [ ]:
--Highest increase in wage
WITH annual_change AS(
    SELECT Year, OCC_CODE, Annually_Mean,
           LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Wage_Previous_Year,
           Annually_Mean - LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
    WHERE Pay_Hourly IS NULL
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Annually_Mean,
       annual_change.Wage_Previous_Year, annual_change.Yearly_Difference AS Highest_Wage_Increase
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE
    WHERE annual_change.Yearly_Difference = (SELECT MAX(Yearly_Difference)
                                             FROM annual_change)

In [ ]:
--Highest wage decrease
WITH annual_change AS(
    SELECT Year, OCC_CODE, Annually_Mean,
           LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Wage_Previous_Year,
           Annually_Mean - LAG(Annually_Mean) OVER ( 
                PARTITION BY OCC_CODE
                ORDER BY Year) AS Yearly_Difference
    FROM Employment_Project..Employment
    WHERE Pay_Hourly IS NULL
)
SELECT annual_change.OCC_CODE, jb.Job_Title, annual_change.Annually_Mean,
       annual_change.Wage_Previous_Year, annual_change.Yearly_Difference AS Highest_Wage_Decrease
FROM annual_change
JOIN Employment_Project..Job_type jb
    ON annual_change.OCC_CODE = jb.OCC_CODE
    WHERE annual_change.Yearly_Difference = (SELECT MIN(Yearly_Difference)
                                             FROM annual_change)